<a href="https://colab.research.google.com/github/Siddhantg2002/Basic-Concepts-2/blob/main/LinearRegressionTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEPS


1.   Import the data and required modules

 (Training/testing datasets if imported seperately then ignore step 3.)

2.   Clean the data (DATA PREPROCESSING):-
*   Seperating Input and Output (to be predicted).
*   Selecting unique values.
*   Ignoring Null Values.
*   Converting Categorical Column into Numerical using Feature Columns.
*   Using input Function to  break the data(dataframe) into epochs and into  batches to feed our model.


3.  Split the data into testing/training sets.
4.   Create a Model.

5.   Train the Model.
6.  Make Predictions.
7. Evaluate and improve untill you achieve higher accuracy.


















In [ ]:
!pip install -q sklearn

  Preparing metadata (setup.py) ... done


In [ ]:
%tensorflow_version 2.x
from  __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc
from six.moves import urllib
import matplotlib.pyplot as plt 
from IPython.display import clear_output


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Training vs Testing Data
You may have noticed that we loaded two different datasets below. This is because when we train models we need two sets of data:
training and testing.
The training data is what we feed to the model so that it can develop and learn. It is usually a much larger size than the testing data.
The testing data is what we use to evaulate the model and see how well it is performing. We must use a seperate set of data that the model has not been trained on to evaluate it. Can you think of why this is?
Well, the point of our model is to be able to make predictions on NEW data, data that we have never seen before. If we simply test the model on the data that it has already seen we cannot measure its accuracy accuratly. We can't be sure that the model hasn't simply memorized our training data. This is why we need our testing and training data to be seperate.

In [ ]:
#LOADING TRAINING DATA
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')# training data
print(dftrain.head())
print('\n\n')
dftrain.describe()


   survived     sex   age  n_siblings_spouses  parch     fare  class     deck  \
0         0    male  22.0                   1      0   7.2500  Third  unknown   
1         1  female  38.0                   1      0  71.2833  First        C   
2         1  female  26.0                   0      0   7.9250  Third  unknown   
3         1  female  35.0                   1      0  53.1000  First        C   
4         0    male  28.0                   0      0   8.4583  Third  unknown   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
2  Southampton     y  
3  Southampton     n  
4   Queenstown     y  





,survived,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.387560,29.631308,0.545455,0.379585,34.385399
std,0.487582,12.511818,1.151090,0.792999,54.597730
min,0.000000,0.750000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,0.000000,7.895800
50%,0.000000,28.000000,0.000000,0.000000,15.045800
75%,1.000000,35.000000,1.000000,0.000000,31.387500
max,1.000000,80.000000,8.000000,5.000000,512.329200


In [ ]:
#LOADING TESTING DATA
dftest = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
print(dftest.head())
print('\n\n')
dftest.describe()


   survived     sex   age  n_siblings_spouses  parch     fare   class  \
0         0    male  35.0                   0      0   8.0500   Third   
1         0    male  54.0                   0      0  51.8625   First   
2         1  female  58.0                   0      0  26.5500   First   
3         1  female  55.0                   0      0  16.0000  Second   
4         1    male  34.0                   0      0  13.0000  Second   

      deck  embark_town alone  
0  unknown  Southampton     y  
1        E  Southampton     y  
2        C  Southampton     y  
3  unknown  Southampton     y  
4        D  Southampton     y  





,survived,age,n_siblings_spouses,parch,fare
count,264.000000,264.000000,264.000000,264.000000,264.000000
mean,0.375000,28.720985,0.469697,0.386364,27.023880
std,0.485042,14.157538,0.978393,0.837775,34.973108
min,0.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,21.000000,0.000000,0.000000,7.925000
50%,0.000000,28.000000,0.000000,0.000000,13.250000
75%,1.000000,35.250000,1.000000,0.000000,27.900000
max,1.000000,74.000000,8.000000,6.000000,263.000000


In [ ]:
#SEPERATING INPUT AND OUTPUT
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')# training data
dftest = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train=dftrain.pop('survived')
y_test=dftest.pop('survived')
print(dftrain.head())
print(dftrain.shape)
print('\n\n')
print(dftest.head())
print(dftest.shape)

      sex   age  n_siblings_spouses  parch     fare  class     deck  \
0    male  22.0                   1      0   7.2500  Third  unknown   
1  female  38.0                   1      0  71.2833  First        C   
2  female  26.0                   0      0   7.9250  Third  unknown   
3  female  35.0                   1      0  53.1000  First        C   
4    male  28.0                   0      0   8.4583  Third  unknown   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
2  Southampton     y  
3  Southampton     n  
4   Queenstown     y  
(627, 9)



      sex   age  n_siblings_spouses  parch     fare   class     deck  \
0    male  35.0                   0      0   8.0500   Third  unknown   
1    male  54.0                   0      0  51.8625   First        E   
2  female  58.0                   0      0  26.5500   First        C   
3  female  55.0                   0      0  16.0000  Second  unknown   
4    male  34.0                   0      0  13.0000  Second    

# Feature Column
In our dataset we have two different kinds of information: Categorical and Numeric
Our categorical data is anything that is not numeric! For example, the sex column does not use numbers, it uses the words "male" and
"female".
Before we continue and create/train a model we must covet our categorical data into numeric data. We can do this by encoding each
category with an integer (ex. male = 1, female = 2).
Fortunately for us TensorFlow has some tools to help!

In [ ]:
#LINEAR ESTIMATOR
CATEGORICAL_COLUMNS = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERIC_COLUMNS = ['age','fare']
feature_columns =[]
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain [feature_name].unique () # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [ ]:
dftrain['embark_town'].unique()

array(['Southampton', 'Cherbourg', 'Queenstown', 'unknown'], dtype=object)

# The Training Process
So we are almost done preparing our dataset and I feel as though it's a good time to explain how our model is trained. Specifically how input data is fed to our model.
For this specific model data is going to be streamed into it in small batches of 32. This means we will not feed the entire dataset to our model at once, but simply small batches of entries.
 We will actually feed these batches to our model multiple times according to the number of epochs.

An epoch is simply one stream of our entire dataset. The number of epochs we define is the amount of times our model will see the entire dataset.
We use multiple epochs in hope that after seeing the same data multiple times the model will better determine how to estimate it.

Ex. if we have 10 ephocs, our model will see the same dataset 10 times.
Since we need to feed our data in batches and multiple times we need to create something called an input function. The input function simply defines how our dataset will be converted into batches at each epoch.


In overfitting, the model was trying to learn everything(including noise data) and in underfitting, the machine learning model is not able to capture the underlying trend of the data

# Input Function
An input function basically is a function used to break the data(dataframe) into epochs and into batches to feed our model.

The TensorFlow model we are going to use requires that the data we pass it comes in as a tf. data. Dataset object. This means we must create a input function that can convert our current pandas dataframe into that object.
Below you'll see a seemingly complicated input function, this is straight from the TensorFlow documentation (https://www.tensorflow.org/tutorials/estimator/linear). 

I've commented as much as I can to make it understandble, but you may want to refer to the documentation for a detailed explination of each method.

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function(): # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle (1000) # randomize order of data
    ds = ds.batch(batch_size). repeat (num_epochs) # split dataset into batches of 32 and repeat process for number of epochs
    return ds # return a batch of the dataset return input_function
  return input_function # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train) # here we will call the input_|function that was returned to us to get a dataset object we can feed our model
test_input_fn = make_input_fn(dftest, y_test, num_epochs=1, shuffle=False)

# Creating the Model
In this tutorial we are going to use a linear estimator to utilize the linear regression algorithm.

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

# Training the Model

In [ ]:
linear_est.train(train_input_fn) # train
result = linear_est.evaluate(test_input_fn) # get model metrics/stats by testing on tetsing data
clear_output () # cleans consoke output
print(result['accuracy'])
# the result variable is simply a dict of stats about our model

0.75


In [ ]:
#PREDICTIONS
result = list(linear_est.predict(test_input_fn))
print(dftest.loc[10])
print(y_test.loc[10])
print(result[10]['probabilities'][1])

sex                       female
age                         28.0
n_siblings_spouses             0
parch                          0
fare                        7.75
class                      Third
deck                     unknown
embark_town           Queenstown
alone                          y
Name: 10, dtype: object
1
0.761485
